### Import Libraries

In [17]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt

### Import Dataset

In [18]:
df = pd.read_csv("weather_sample.csv")

### COUNTS

In [19]:
df

,Outlook,Temperature,Humidity,Windy,Play
0,overcast,hot,high,False,yes
1,overcast,cool,normal,True,yes
2,overcast,mild,high,True,yes
3,overcast,hot,normal,False,yes
4,rainy,mild,high,False,yes
5,rainy,cool,normal,False,yes
6,rainy,cool,normal,True,no
7,rainy,mild,normal,False,yes
8,rainy,mild,high,True,no
9,sunny,hot,high,False,no


In [20]:
df.groupby('Play').describe()

Outlook                       Temperature                   Humidity  \
       count unique       top freq       count unique   top freq    count   
Play                                                                        
no         5      2     sunny    3           5      3  mild    2        5   
yes        9      3  overcast    4           9      3  mild    4        9   

                         Windy                     
     unique     top freq count unique    top freq  
Play                                               
no        2    high    4     5      2   True    3  
yes       2  normal    6     9      2  False    6

## BY USING GINI INDEX

In [73]:
def calc_gini_impurity(input_df, output_df):    
    df_col = pd.crosstab(input_df, output_df)
    df_col['total'] = df_col['no'] + df_col["yes"]
    df_col['prob_no'] = df_col['no'] / df_col['total']
    df_col['prob_yes'] = df_col['yes'] / df_col['total']
    df_col['Gini_Index'] = 1 - (df_col['prob_yes'] ** 2 + df_col['prob_no'] ** 2)
    df_col['average_weighted'] = (df_col['total']/df_col['total'].sum()) * df_col['Gini_Index']
    return df_col

In [74]:
outlook = calc_gini_impurity(df.Outlook,df.Play)
outlook

Play,no,yes,total,prob_no,prob_yes,Gini_Index,average_weighted
Outlook,,,,,,,
overcast,0,4,4,0.0,1.0,0.00,0.000000
rainy,2,3,5,0.4,0.6,0.48,0.171429
sunny,3,2,5,0.6,0.4,0.48,0.171429


In [75]:
temp = calc_gini_impurity(df.Temperature,df.Play)
temp

Play,no,yes,total,prob_no,prob_yes,Gini_Index,average_weighted
Temperature,,,,,,,
cool,1,3,4,0.250000,0.750000,0.375000,0.107143
hot,2,2,4,0.500000,0.500000,0.500000,0.142857
mild,2,4,6,0.333333,0.666667,0.444444,0.190476


In [1]:
wind = calc_gini_impurity(df.Windy,df.Play)
wind

NameError: name 'calc_gini_impurity' is not defined

In [2]:
humd = calc_gini_impurity(df.Humidity,df.Play)
humd

NameError: name 'calc_gini_impurity' is not defined

In [78]:
print("OUTLOOK:", outlook.average_weighted.sum())
print("TEMP:", temp.average_weighted.sum())
print("HUMD:", humd.average_weighted.sum())
print("WINDY:", wind.average_weighted.sum())

OUTLOOK: 0.34285714285714286
TEMP: 0.44047619047619047
HUMD: 0.3673469387755103
WINDY: 0.42857142857142855


CHOOSE MINIMUM => WE CHOOSE OUTLOOK AS ROOT NODE

OVERCAST -> yes (100%)
RAINY -> 

### SUBNODE => RAINY

In [140]:
df_outlook_rainy = df.loc[df['Outlook'] == 'rainy']
df_outlook_rainy.rename = "RAINY"
calc_gini_impurity(df_outlook_rainy.Temperature,df_outlook_rainy.Play)

Play,no,yes,total,prob_no,prob_yes,Gini_Index,average_weighted
Temperature,,,,,,,
cool,1,1,2,0.500000,0.500000,0.500000,0.200000
mild,1,2,3,0.333333,0.666667,0.444444,0.266667


In [145]:
calc_gini_impurity(df_outlook_rainy.Humidity,df_outlook_rainy.Play)

Play,no,yes,total,prob_no,prob_yes,Gini_Index,average_weighted
Humidity,,,,,,,
high,1,1,2,0.500000,0.500000,0.500000,0.200000
normal,1,2,3,0.333333,0.666667,0.444444,0.266667


In [146]:
calc_gini_impurity(df_outlook_rainy.Windy,df_outlook_rainy.Play)

Play,no,yes,total,prob_no,prob_yes,Gini_Index,average_weighted
Windy,,,,,,,
False,0,3,3,0.0,1.0,0.0,0.0
True,2,0,2,1.0,0.0,0.0,0.0


### SUBNODE => SUNNY

In [149]:
df_outlook_sunny = df.loc[df['Outlook'] == 'sunny']

calc_gini_impurity(df_outlook_sunny.Temperature,df_outlook_sunny.Play)

Play,no,yes,total,prob_no,prob_yes,Gini_Index,average_weighted
Temperature,,,,,,,
cool,0,1,1,0.0,1.0,0.0,0.0
hot,2,0,2,1.0,0.0,0.0,0.0
mild,1,1,2,0.5,0.5,0.5,0.2


In [150]:
calc_gini_impurity(df_outlook_sunny.Humidity,df_outlook_sunny.Play)

Play,no,yes,total,prob_no,prob_yes,Gini_Index,average_weighted
Humidity,,,,,,,
high,3,0,3,1.0,0.0,0.0,0.0
normal,0,2,2,0.0,1.0,0.0,0.0


In [151]:
calc_gini_impurity(df_outlook_sunny.Windy,df_outlook_sunny.Play)

Play,no,yes,total,prob_no,prob_yes,Gini_Index,average_weighted
Windy,,,,,,,
False,2,1,3,0.666667,0.333333,0.444444,0.266667
True,1,1,2,0.500000,0.500000,0.500000,0.200000


## CHI SQUARED TEST

In [32]:
def chi_square(input_df, output_df):
    df_chi = pd.crosstab(input_df, output_df) # GET TABULAR
    # EXPECTED (AVERAGE)
    df_chi['exp_no'] = df_chi['exp_yes'] = (df_chi['no'] + df_chi['yes'])/2
    # DEVIATTION
    df_chi['chi_yes'] = ((df_chi["yes"] - df_chi['exp_yes']) ** 2) / df_chi['exp_yes']
    df_chi['chi_no'] = ((df_chi["no"] - df_chi['exp_no']) ** 2 ) / df_chi['exp_no']
    return sum(df_chi['chi_yes'] + df_chi['chi_no'])

In [33]:
print(chi_square(df.Humidity, df.Play))
print(chi_square(df.Outlook, df.Play))
print(chi_square(df.Windy, df.Play))
print(chi_square(df.Temperature, df.Play))

3.7142857142857144
4.4
2.0
1.6666666666666665


In [34]:
df_outlook_rainy = df.loc[df['Outlook'] == 'rainy']
print(chi_square(df_outlook_rainy.Humidity, df_outlook_rainy.Play))
print(chi_square(df_outlook_rainy.Windy, df_outlook_rainy.Play))
print(chi_square(df_outlook_rainy.Temperature, df_outlook_rainy.Play))

0.3333333333333333
5.0
0.3333333333333333


In [ ]:
df_outlook_sunny = df.loc[df['Outlook'] == 'sunny']
print(chi_square(df_outlook_sunny.Humidity, df_outlook_sunny.Play))
print(chi_square(df_outlook_sunny.Windy, df_outlook_sunny.Play))
print(chi_square(df_outlook_sunny.Temperature, df_outlook_sunny.Play))